# Recipe Check with Azure OpenAI - Tightly Typed Structured Data
This notebook implements a Python solution to match recipes (dishes) with available ingredients, leveraging Azure OpenAI for reasoning and natural language processing.
The solution:
- Accepts a JSON-based input schema for requested dishes and available ingredients.
- Calls Azure OpenAI to evaluate the feasibility of creating each dish.
- Outputs a JSON object indicating whether each dish can be created and lists the required ingredients.

The core capability demonstrated in this notebook is - **Providing a JSON-based output schema**

## Environment Setup
1. Create a `.env` file in the configuration folder with the following contents, we provided a sample file in the same folder:

```plaintext
AOAI_APIKEY = <your key>
AOAI_ENDPOINT = <your endpoint>
EMBEDDING_DEPLOYMENTNAME = <your embedding deployment name>
CHATCOMPLETION_DEPLOYMENTNAME = <your chat completion deployment name>
```
2. Install necessary libraries:



In [ ]:
! pip install openai==1.59.3 python-dotenv

In [38]:
def check_azure_openai_config(subscription_key: Optional[str], endpoint: Optional[str], deployment: Optional[str]) -> None:
    """
    Checks and prints the status of Azure OpenAI configuration variables.

    Args:
        subscription_key (Optional[str]): Azure OpenAI API key.
        endpoint (Optional[str]): Azure OpenAI endpoint URL.
        deployment (Optional[str]): Azure OpenAI deployment name.
    """
    config_checks = {
        "Azure OpenAI API key": subscription_key,
        "Azure OpenAI endpoint": endpoint,
        "Azure OpenAI deployment name": deployment,
    }

    for config_name, config_value in config_checks.items():
        if config_value is None:
            print(f"{config_name} is not set")
        else:
            print(f"{config_name} is set")

In [ ]:
# Load environment variables
import os
from dotenv import load_dotenv

# Load .env file
load_dotenv("../configuration/.env")

# Azure OpenAI environment variables
subscription_key = os.getenv("AOAI_APIKEY")
endpoint = os.getenv("AOAI_ENDPOINT")
deployment = os.getenv("CHATCOMPLETION_DEPLOYMENTNAME")
# check if any of the variables are not set and output a message

check_azure_openai_config(subscription_key, endpoint, deployment)


## Input and output classes

Now lets define the input and output schemas for our recipe-checking application using Python classes. These classes are built with the _pydantic_ library to provide:
1.	Structure: Clear definitions of the fields required for input and output, ensuring consistency across the application.
2.	Validation: Automatic type checking and validation of data at runtime, reducing potential errors caused by invalid inputs.
3.	Serialization: Easy conversion between JSON and Python objects, simplifying data exchange with external systems like APIs.
4.	Documentation: Enhanced readability and maintainability with detailed descriptions for each attribute.

The schema consists of:
- Input Schema: Represents the list of dishes to check and the available ingredients.
- Output Schema: Represents the results for each dish, indicating whether it can be created and the list of required ingredients.

These classes will ensure the integrity of data as it flows through the system and serve as the foundation for interacting with Azure OpenAI to process the input and generate the output.

In [40]:
from typing import List, Optional
from pydantic import BaseModel, Field

class DishRequest(BaseModel):
    """
    A class representing a requested dish.

    Attributes:
        dish_name: Name of the requested dish.
    """
    dish_name: str = Field(
        description="Name of the requested dish, e.g., Spaghetti Bolognese"
    )


class RecipeCheckInput(BaseModel):
    """
    A class representing the input for recipe checking.

    Attributes:
        requested_dishes: List of dishes to check for feasibility.
        available_ingredients: List of ingredients available in the kitchen.
    """
    requested_dishes: List[DishRequest] = Field(
        description="List of dishes to check for feasibility."
    )
    available_ingredients: List[str] = Field(
        description="List of ingredients available in the kitchen, e.g., onions, garlic."
    )


class DishAvailability(BaseModel):
    """
    A class representing the availability status of a dish.

    Attributes:
        dish_name: Name of the dish being checked.
        can_be_created: Indicates whether the dish can be created with available ingredients.
        required_ingredients: List of ingredients required for the dish (both available and missing).
    """
    dish_name: str = Field(
        description="Name of the dish being checked, e.g., Spaghetti Bolognese."
    )
    can_be_created: bool = Field(
        description="Indicates whether the dish can be created with available ingredients."
    )
    required_ingredients: List[str] = Field(
        description="List of ingredients required for the dish, both available and missing."
    )


class RecipeCheckOutput(BaseModel):
    """
    A class representing the output for recipe checking.

    Attributes:
        dish_availability: List of availability information for each requested dish.
    """
    dish_availability: List[DishAvailability] = Field(
        description="List of availability information for each requested dish."
    )

## Azure OpenAI Integration

Next is the integration with Azure OpenAI to enable the processing of structured inputs and outputs. This integration utilizes the AzureOpenAI client from the openai library, initialized with key-based authentication. (EntraID authentication is recommended for production scenarios.)

Key Features of the Integration
- **Structured Outputs**: This integration leverages the capability introduced in API version 2024-08-01-preview, which allows for defining a structured output schema directly in API calls. This feature ensures that the model’s responses adhere to a specific JSON schema, simplifying downstream processing and improving reliability.

### Code Implementation

The following code initializes the Azure OpenAI client with the required configuration:


In [33]:
import os  
import base64
from openai import AzureOpenAI  

# Initialize Azure OpenAI client with key-based authentication    
client = AzureOpenAI(  
    azure_endpoint=endpoint,  
    api_key=subscription_key,  
    api_version="2024-08-01-preview",
)
    

### Documentation Reference

This implementation is based on the structured output feature explained in the Azure OpenAI [documentation](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/structured-outputs?tabs=python-secure). By specifying a structured schema in the API call, we ensure that the model’s responses conform to a predefined format, reducing the need for manual parsing or validation.

This version (2024-08-01-preview) introduces advanced capabilities like schema-guided outputs, making it particularly useful for applications requiring high consistency and accuracy in generated data.


## Use Case: Recipe Check

The prompt below is a rudimentary example of a recipe-checking scenario, as the purpose of the example is not to find the right ingredients but to demonstrate the structured output feature.

Below is a system message that performs the recipe check for given set of dishes and ingredients.

In [34]:
system_message = "You are a cooking assistant that determines whether requested dishes can be created based on the list of available ingredients. If an ingredient for the dish is missing, it cannot be created."

## Helper Function : generate_prompt

This method, generate_prompt, constructs the input prompt for Azure OpenAI by extracting information from a tightly typed RecipeCheckInput object. The prompt includes:
- A list of requested dishes, derived from the requested_dishes field.
- A list of available ingredients, extracted from the available_ingredients field.

Key Features
- Tightly Typed Input:
   - The method accepts a RecipeCheckInput object, a Pydantic model that ensures the input adheres to a predefined schema. This guarantees consistency, as the fields requested_dishes and available_ingredients are validated at runtime.
   - By using tightly typed inputs, we enforce strong guarantees on data quality, reducing errors and improving the reliability of the prompt generation process.
- Readable and Structured Prompt:
   - The generated prompt is simple and clear, listing the requested dishes and available ingredients in a human-readable format. This ensures that the AI can interpret the task effectively.

In [35]:
def generate_prompt(input_data: RecipeCheckInput) -> str:
    return (
        f"Requested Dishes: {[dish.dish_name for dish in input_data.requested_dishes]}\n"
        f"Available Ingredients: {input_data.available_ingredients}\n"
    )

**Input**: A RecipeCheckInput object containing:

- requested_dishes: A list of DishRequest objects, each specifying a dish name.
- available_ingredients: A list of strings representing the ingredients available in the kitchen.

**Output**: A string formatted to describe the requested dishes and available ingredients.

**Why the Use Tightly Typed Input?**

By defining and enforcing a schema through Pydantic, we ensure:
- Input consistency: Only data matching the RecipeCheckInput structure can be passed to the method.
- Automatic validation: Fields are checked for correctness (e.g., data types) at runtime.
- Enhanced clarity: The schema serves as a form of documentation, making the expected input format immediately clear.

This method is a critical step in ensuring the AI receives a well-structured, context-rich prompt, enabling accurate and reliable output generation.

## Helper Function : process_recipe_check

Here we combine the Azure OpenAI integration with the recipe-checking logic to evaluate the feasibility of creating each dish based on the available ingredients. The method process_recipe_check performs the following steps:

In [36]:
def check_recipes(input_data: RecipeCheckInput) -> RecipeCheckOutput:
    """
    Calls Azure OpenAI to match dishes with available ingredients and generates the output.

    Args:
        input_data (RecipeCheckInput): Input data object.

    Returns:
        RecipeCheckOutput: Output data object with dish availability information.
    """
    # Generate the prompt
    prompt = generate_prompt(input_data)

    # Prepare messages for the chat model
    messages = [{"role":"system", "content":system_message},{"role": "user", "content": prompt}]

    # Call Azure OpenAI
    completion = client.beta.chat.completions.parse(
        model=deployment,
        messages=messages,
        response_format=RecipeCheckOutput,  # Directly specify the response schema as a Python class
        max_tokens=800,
        temperature=0.7,
        top_p=0.95   
    )

    # `completion` is automatically parsed as a RecipeCheckOutput object
    return completion

## Sample use

The following is an example of how to call the newly created method with specific input request.


	🛑 Important: While this notebook aims to check recipe feasibility based on ingredients, do not attempt to cook solely based on the results. The feasibility of the output also depends on your cooking skills! Remember, even the best ingredients can’t save a bad cook. 😄

In [ ]:
# Example input
example_input = RecipeCheckInput(
    requested_dishes=[
        DishRequest(dish_name="Spaghetti Bolognese"),
        DishRequest(dish_name="Carnitas"),
    ],
    available_ingredients=[
        "carrots", "onions", "bell peppers", "garlic", "pork", "spaghetti", "tomato sauce"
    ],
)

# Call the method
example_output = check_recipes(example_input)

# Display output
print(example_output.choices[0].message.parsed.json(indent=2))